In [1]:
# https://steemit.com/python/@marketstack/how-to-download-historical-price-data-from-binance-with-python
import requests        # for making http requests to binance
import json            # for parsing what binance sends back to us
import pandas as pd    # for storing and manipulating the data we get back
import numpy as np     # numerical python, i usually need this somewhere 
                       # and so i import by habit nowadays

import matplotlib.pyplot as plt # for charts and such
    
import datetime as dt  # for dealing with times

In [2]:
def get_bars(symbol, interval, start_time, end_time, limit=1000):
    url = 'https://api.binance.com/api/v1/klines'
    
    start_time = str(int(start_time.timestamp() * 1000))
    end_time = str(int(end_time.timestamp() * 1000))
    
    params = {'symbol': symbol,
             'interval': interval,
             'startTime': start_time,
             'endTime': end_time,
             'limit': str(limit)}
    
    data = json.loads(requests.get(url, params=params).text)
    df = pd.DataFrame(data)
    
    df.columns = ['open_time',
                 'open', 'high', 'low', 'close', 'v',
                 'close_time', 'qav', 'num_trades',
                 'taker_base_vol', 'taker_quote_vol', 'ignore']
    
    df.index = [dt.datetime.fromtimestamp(x/1000.0) for x in df.close_time]
    
    #Getting rid of last row which contains data from the following day
    df = df.iloc[:-1,:]
    
    return df

In [3]:
def get_historical_data(symbol, interval, start_time, end_time, limit=1000):
    delta = end_time - start_time # as timedelta
    # Store results for each day here
    dataframes = []
    
    for i in range(delta.days + 1):
        start_day = start_time + dt.timedelta(days=i)
        end_day = start_day + dt.timedelta(days=1)
        # 
        day_bars = get_bars(symbol, interval, start_day, end_day, limit)
        
        dataframes.append(day_bars)
    # Concat all day dataframes in a single dataframe
    return pd.concat(dataframes)
        

In [4]:
start_time = dt.datetime(2020,1,1)
end_time = dt.datetime(2020,1,5)

all_days = get_historical_data('ETHUSDT', '5m', start_time, end_time)

all_days.head()

,open_time,open,high,low,close,v,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
2020-01-01 00:04:59.999,1577833200000,129.05000000,129.18000000,128.86000000,129.00000000,447.11874000,1577833499999,57713.28737640,319,275.88227000,35612.18745840,0
2020-01-01 00:09:59.999,1577833500000,129.03000000,129.35000000,129.00000000,129.00000000,339.21662000,1577833799999,43809.13643490,225,151.59562000,19580.74978810,0
2020-01-01 00:14:59.999,1577833800000,129.03000000,129.27000000,129.01000000,129.15000000,191.20956000,1577834099999,24704.32736070,188,73.24096000,9462.57059010,0
2020-01-01 00:19:59.999,1577834100000,129.20000000,129.27000000,128.84000000,129.08000000,469.76125000,1577834399999,60651.76096350,323,205.17778000,26483.01880880,0
2020-01-01 00:24:59.999,1577834400000,129.10000000,129.19000000,128.89000000,128.95000000,172.54920000,1577834699999,22267.10610910,180,79.72585000,10289.36867900,0


In [6]:
all_days.tail()

,open_time,open,high,low,close,v,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
2020-01-05 23:39:59.999,1578263700000,134.79000000,134.86000000,134.61000000,134.71000000,648.67922000,1578263999999,87399.48124910,394,325.64688000,43882.38713260,0
2020-01-05 23:44:59.999,1578264000000,134.71000000,135.02000000,134.71000000,134.95000000,282.72097000,1578264299999,38142.34193500,255,216.53484000,29212.69319390,0
2020-01-05 23:49:59.999,1578264300000,134.97000000,134.97000000,134.68000000,134.91000000,437.29627000,1578264599999,58950.97190700,339,280.88861000,37867.69908500,0
2020-01-05 23:54:59.999,1578264600000,134.89000000,135.08000000,134.85000000,135.04000000,611.57372000,1578264899999,82556.43497470,345,302.24628000,40798.02448260,0
2020-01-05 23:59:59.999,1578264900000,135.04000000,135.07000000,134.96000000,135.04000000,329.91646000,1578265199999,44537.70926220,245,210.08441000,28361.23838440,0


In [5]:
len(all_days)

1440